In [3]:
from trnasimtools.serialize import SerializeTwoCodonMultiTranscript
import os

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [2]:
def read_sim(path_pref, seed_start=1, seed_end=3, seed_incr=1, time_limit=None):
    """ 
    Reads in output for an arbitrary number of simulation trials 
    (with different seeds) and concatenates results into a single
    dataset.
    """
    df_master = pd.read_csv(f"{path_pref}_{seed_start}.tsv", sep="\t")
    df_master["seed"] = str(seed_start)
    for i in range(seed_start+1, seed_end+1):
        tmp = pd.read_csv(f"{path_pref}_{i}.tsv", sep="\t")
        tmp["seed"] = str(i)
        df_master = df_master.append(tmp, ignore_index=True)
    if time_limit is not None:
        df_master = df_master[df_master.time < time_limit]
    return df_master
    
def concat_sims(path_pref, max_seed):
    df_master = pd.read_csv(f"{path_pref}_1.tsv", sep="\t")
    df_master["seed"] = str(1)
    for i in range(2, max_seed+1):
        tmp = pd.read_csv(f"{path_pref}_{i}.tsv", sep="\t")
        tmp["seed"] = str(i)
        df_master = df_master.append(tmp, ignore_index=True)
    
    return df_master
    
def get_average_protein(path, perc_good, time, max_seed):
    df_master = concat_sims(path, max_seed)
    df_master["time"] = df_master["time"].apply(np.ceil)
    tmp = df_master.groupby(["time", "species"])["protein", "ribo_density"].mean().reset_index()
    tmp = tmp[(tmp.time > time - 50)] # look at simulations after equilibrium is reached
    tmp["ribo_density_ave"] = tmp[(tmp.species == "cellularProtein")]['ribo_density'].mean()
    tmp = tmp[(tmp.time == time)]
    return tmp

In [7]:
# simulation parameters 
time_limit = 100
time_step = 5
transcript_lens = [1000, 300]
cellular_transcript_copy_number = 100
gfp_transcript_copy_number = 20
ribosome_copy_number = 500
total_trna = 2500
ecol_rbs_rate = 100000.0
ribosome_binding_rates = [10000.0, 30000.0, 100000.0, 300000.0, 1000000.0, 3000000.0]
trna_charging_rates = [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0, 10000.0]
transcript_names = ["cellularProtein", "GFP"]
trna_composition = [(0.7, 0.3)]
gfp_mrna_compositions = [(0.9, 0.1), (0.7, 0.3), (0.5, 0.5), (0.3, 0.7), (0.1, 0.9)]
ribosome_speed = 0.5
ribosome_footprint = 15

ecol_mrna_compositions = [(0.2, 0.8), (0.4, 0.6), (0.6, 0.4), (0.7, 0.3), (0.8, 0.2), (1.0, 0)]
date = "may-28-2024"

In [9]:
!mkdir ../yaml/may-28-2024

In [10]:
scratch_dir = "/scratch/07227/hilla3/output"

for trna_prop in trna_composition:
    for comp in gfp_mrna_compositions:
        for ecol in ecol_mrna_compositions:
            serializer = SerializeTwoCodonMultiTranscript(transcript_lens=transcript_lens,
                                                           codon_comps=[ecol, comp],
                                                           trna_proportion=trna_prop,
                                                           transcript_names=transcript_names,
                                                           time_limit=time_limit,
                                                           time_step=time_step)
            serializer.serialize(f"../yaml/{date}")
        
configs = os.listdir(f"../yaml/{date}")
with open(f"../tacc/launcher/{date}.txt", "w") as stream:
    for config in configs:
        for binding_rate in ribosome_binding_rates:
            for charging_rate in trna_charging_rates:
                for seed in range(1, 4):
                    cmd = f"python3 ./scripts/twocodonmultitranscript.py ./yaml/{date}/{config} {seed} {cellular_transcript_copy_number} {gfp_transcript_copy_number} " + \
                      f"{ribosome_copy_number} {total_trna} {ecol_rbs_rate} {binding_rate} {charging_rate} {charging_rate} " + \
                      f"{scratch_dir}/{date} {ribosome_speed} {ribosome_footprint}"
                    stream.write(cmd)
                    stream.write("\n")